In [2]:
import pandas as pd
import numpy as np
from pylab import *

import re
import os
import glob
import math
from pathlib import Path
from scipy import stats
from scipy.spatial.distance import pdist, squareform
from scipy.signal import savgol_filter
from openpyxl import load_workbook
import seaborn as sns
from matplotlib import pyplot as plt
import peakutils

def distance(data): # generate distance matrix
    b = pdist(data)
    pattern = squareform(b)
    d = pd.DataFrame(pattern)
    return d

def moving_average(x, w):                          #'w' is the smoothing level,'x' is the data required to be smoothed
    return np.convolve(x, np.ones(w), 'valid') / w


In [12]:
try:
    os.mkdir('output') # create file named "output"
except:
    print('Directory \'output\' already exists.')

for file_name in glob.glob('*signal.xlsx'):
    base_name = Path(file_name).stem
    print(base_name)
    df = pd.read_excel(file_name, sheet_name=0)
    nNeuron = len(df.iloc[0,:])-4       # number of neurons
    for i in range(0,nNeuron-1):
        if pd.isna(df.iloc[i,1]) == True:
            df.iloc[i,1] = 'n'+str(i+1)
        elif str(df.iloc[i,1]).isspace() == True:
            df.iloc[i,1] = 'n'+str(i+1)
    nameList = df.iloc[0:nNeuron,1].tolist()
    df_nameList = pd.DataFrame(nameList).T
    writer = pd.ExcelWriter(f'output/{base_name}_near_smooth_distan_corre_deri.xlsx', engine="xlsxwriter")
    # df = pd.read_excel('File18_20210701YR_OH16230_death-10sInt.xlsx')
    """
    get distance
    """
    df1=df.iloc[:,0:3]
    df1.dropna(thresh = 2, inplace = True)        
    xy = []
    for i in df1.iloc[:,2]:
        xy.append(re.findall('\d+',i))
    xy = np.array(xy)
    xyP =np.delete(xy, np.s_[:1:1], 1)
    dis = distance(float64(xyP))                 # get distance between each two neurons

    """
    smooth
    """
    df2 = df.iloc[:,4:]
    data = []
    for i in df2.keys()[0:]:
        data.append(savgol_filter(df2[i], 71, 10))    
    dff = pd.DataFrame(data).T
    dff.to_excel(writer, sheet_name='smoo_savgol', header=nameList)

    """
    moving average
    """
    smooth=pd.DataFrame(list(dff.index))
    # smooth=pd.DataFrame(moving_average(df.iloc[:,1],2),columns=['S1'])
    for i in dff.keys()[0:]:
        smooth[i]=pd.DataFrame(moving_average(dff[i],15))
    smooth = smooth.dropna()
    smooth1 = smooth.dropna()
    smooth.to_excel(writer, sheet_name='smoo_aver', header=nameList)

    #smooth2 = dff.rolling(5, center=True, min_periods=1).mean()
    #smooth2.to_excel('smoothed.xlsx', index=False, header=None)

    """
    get correlations
    """
    correlations = dff.corr()

    dis.to_excel(writer, sheet_name='distance', header=nameList)
    correlations.to_excel(writer, sheet_name='correlation', header=nameList)

    """
    get data point for each neuron pair### (x,y), x = distance between one pair of neuron; 
    y = correlation between one pair of neuron
    """
    dist = []
    for i in dis.keys():
        if i == 97:
            break
        dist = dist + list(dis.iloc[i+1:,i])

    correl = []
    for i in correlations.keys():
        if i == 97:
            break
        correl = correl + list(correlations.iloc[i+1:,i])

    spss = pd.DataFrame([dist,correl]).T
    spss.to_excel(writer, sheet_name='dis_corr_datapair',index=False)

    """
    derivative
    """
    derivative = pd.DataFrame(smooth.rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0]))/0.133333

    derivative.to_excel(writer, sheet_name='deri',index=False, header=nameList) 
    
    
    writer1 = pd.ExcelWriter(f'output/{base_name}_near_peak_baseline.xlsx', engine="xlsxwriter")
    """
    get peak value and index
    """ 
    peak_p = []
    peak_s = []
    base = []
    bb = 0
    for i in smooth.keys():
        data1 = smooth.iloc[20:,i]
        
        x_peak = data1.idxmax()      
        y_peak = data1.max()
        peak_p.append([x_peak,y_peak])
        """
        get baseline
        """
        baseline_values = peakutils.baseline(smooth.iloc[:,i])
        baseline = np.average(baseline_values)
        base.append(baseline_values)

        """
        get start of peak
        """    
        p_data = smooth.iloc[: x_peak + 1, i][::-1] #[::-1] can reverse the order of row & p_data.iloc[:, ::-1] or p_data.columns[::-1] reverses the order of the DataFrame's sequence of columns
        for m in p_data.keys():
        #     print(p_data[i])
            standard = baseline_values[m]
            if p_data[m] < standard:
                x_s = m
                y_s = p_data[m]
                peak_s.append([x_s,y_s])
                break
            elif m == 0:
                peak_s.append([0,0])
                print(0)
            else:
                continue
    #     print(x_l,y_l)    

    base =  pd.DataFrame(base).T
    peak_p = pd.DataFrame(peak_p)
    peak_s = pd.DataFrame(peak_s)
    base.to_excel(writer, sheet_name='baseline', header=nameList) 
    """
    get true baseline
    """
    baseline_l = []
    for i in smooth.keys():
        l_dex = int(peak_s.iloc[i,0])
        if l_dex == 0:
            baseline_values = peakutils.baseline(smooth.iloc[:,i])
            baseline = np.average(baseline_values)
        else:
            new_d = smooth.iloc[:l_dex,i]
            baseline_values = peakutils.baseline(new_d)
            baseline = np.average(baseline_values)
        baseline_l.append(baseline)    
        smooth1.iloc[:,i] = (smooth1.iloc[:,i] - baseline)/baseline
    baseline_l = pd.DataFrame(baseline_l)
    baseline_l.to_excel(writer, sheet_name='baseline_value') 
    smooth1.to_excel(writer, sheet_name='F0_Normoli', header=nameList) 
    writer.close()
    ######################################
    #####after this, should amplitude = peak value
    ######################################
    """
    get true peak
    """
    peak_p2 = []
    peak_s2 = []
    amplit = []
    for i in smooth1.keys():
        data2 = smooth1.iloc[20:,i]
        x_peak2 = data2.idxmax()
        y_peak2 = data2.max()
        peak_p2.append([x_peak2,y_peak2])
        
    peak_p2 = pd.DataFrame(peak_p2,columns = ['x_peak','y_peak'])
    """
    start of peak after normalisation
    """
    peak_s2 = []
    for i in smooth1.keys():
        x_l2 = peak_s.iloc[i,0]
        y_l2 = smooth1.iloc[x_l2,i]
        peak_s2.append([x_l2,y_l2])
    peak_s2 = pd.DataFrame(peak_s2,columns = ['x_start','y_start'])

    x_dis = peak_p2['x_peak']-peak_s2['x_start']
    y_dis = peak_p2['y_peak']-peak_s2['y_start']
    xy_ratio = y_dis/x_dis
    xy_dis = pd.DataFrame([x_dis,y_dis,xy_ratio],index = ['x_distance',"y_distance","xy_ratio"]).T
    
    peak2 = pd.concat([peak_s2,peak_p2,xy_dis], axis = 1)
    peak2.to_excel(writer1, sheet_name='peak') 
    writer1.close()

Directory 'output' already exists.
20221016YR_triple_7%tBuOOH_calcium_signal
0
0
0
0
0
0
0
0
0
0
20211205YR_OH16230_5%tBOOH_calcium_signal
0
0
0
0
0


In [ ]:
"""
test if the index of statr is 0 or 20
"""
qq = 0
for i in peak_s[0]:
    if i == 0 or i == 20:
        print(qq)
    qq += 1